In [1]:
import pandas as pd

In [2]:
yelp=pd.read_csv("yelp.csv")
yelp.head(1)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0


In [3]:
yelp.shape

(10000, 10)

In [4]:
yelp.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [5]:
yelp_best_worst=yelp.loc[(yelp.stars==5)|(yelp.stars==1),:]

In [6]:
yelp_best_worst.shape

(4086, 10)

In [7]:
X=yelp_best_worst.text
y=yelp_best_worst.stars

In [8]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(3064L,)
(1022L,)
(3064L,)
(1022L,)


C:\Users\Karthik\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()

In [10]:
X_train_dtm=vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 16825)

In [11]:
X_test_dtm=vect.transform(X_test)
X_test_dtm.shape

(1022, 16825)

In [12]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [13]:
nb.fit(X_train_dtm,y_train)
y_pred_class=nb.predict(X_test_dtm)

In [15]:
from sklearn import metrics


In [16]:
metrics.accuracy_score(y_test,y_pred_class)

0.91878669275929548

In [17]:
metrics.confusion_matrix(y_test,y_pred_class)

array([[126,  58],
       [ 25, 813]])

In [18]:
#Null Accuracy
y_test.value_counts()
nullAccuracy=y_test.value_counts().head(1)/y_test.shape
print nullAccuracy

5    0.819961
Name: stars, dtype: float64


In [20]:
#False Positive
X_test[y_test<y_pred_class].head(5)

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
Name: text, dtype: object

In [21]:
#False Negative
X_test[y_test>y_pred_class].head(5)

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
Name: text, dtype: object

In [22]:
X_train_tokens=vect.get_feature_names()

In [23]:
print nb.feature_count_.shape
print nb.feature_count_

(2L, 16825L)
[[ 26.   4.   1. ...,   0.   0.   0.]
 [ 39.   5.   0. ...,   1.   1.   1.]]


In [24]:
one_star_count=nb.feature_count_[0,:]
five_star_count=nb.feature_count_[1,:]

In [25]:
tokens=pd.DataFrame({'token':X_train_tokens,'onestar':one_star_count,'fivestar':five_star_count}).set_index('token')
tokens.head()

,fivestar,onestar
token,,
00,39.0,26.0
000,5.0,4.0
00a,0.0,1.0
00am,2.0,3.0
00pm,4.0,1.0


In [26]:
tokens['fivestar']=tokens.fivestar+1
tokens['onestar']=tokens.onestar+1

In [27]:
nb.class_count_

array([  565.,  2499.])

In [28]:
tokens['onestar']=tokens.onestar/nb.class_count_[0]
tokens['fivestar']=tokens.fivestar/nb.class_count_[1]

In [29]:
tokens['fivestarratio']=tokens.fivestar/(tokens.onestar+tokens.fivestar)
tokens.head()

,fivestar,onestar,fivestarratio
token,,,
00,0.016006,0.047788,0.250908
000,0.002401,0.008850,0.213409
00a,0.000400,0.003540,0.101564
00am,0.001200,0.007080,0.144983
00pm,0.002001,0.003540,0.361115


In [30]:
X=yelp.text
y=yelp.stars

In [31]:
y.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [35]:
X_train_dtm=vect.fit_transform(X_train)
X_test_dtm=vect.transform(X_test)

In [36]:
nb.fit(X_train_dtm,y_train)
y_pred_class=nb.predict(X_test_dtm)

In [37]:
metrics.accuracy_score(y_test,y_pred_class)

0.47120000000000001

In [38]:
#Null Accuracy
y.value_counts().head(1)/y.shape

4    0.3526
Name: stars, dtype: float64

In [39]:
print metrics.classification_report(y_test,y_pred_class)

             precision    recall  f1-score   support

          1       0.54      0.30      0.38       185
          2       0.39      0.07      0.12       234
          3       0.29      0.10      0.14       365
          4       0.43      0.71      0.53       884
          5       0.58      0.53      0.55       832

avg / total       0.46      0.47      0.43      2500



In [40]:
metrics.confusion_matrix(y_test,y_pred_class)

array([[ 55,  14,  24,  65,  27],
       [ 28,  16,  41, 122,  27],
       [  5,   7,  35, 281,  37],
       [  7,   0,  16, 629, 232],
       [  6,   4,   6, 373, 443]])

In [44]:
#Precision
precision=55/float(55+28+5+7+6)
print precision

0.544554455446


In [ ]:
#Recall

In [45]:
recall=55/float(55+14+24+65+27)
print recall

0.297297297297


In [47]:
f1score=2*precision*recall/(precision+recall)
f1score

0.38461538461538464

In [48]:
support=55+14+24+65+27
print support

185
